In [841]:
import pandas as pd
import os 
import string
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords 
import enchant

## Helper Functions

In [1035]:
def remove_numbers(list):
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]
    return list

def remove_length(list):
    for object in list:
        if len(object) < 2:
            list.remove(object)
    return list

def length_probs(list,count_normalizer,multipler,prob):
    spam_length_probs = []
    ham_length_probs = []
    counter = 0
    prob_spam = prob * multipler * count_normalizer
    prob_ham = (1 - prob) * multipler
    
    for object in list:
        if len(object)<2:
            spam_length_probs.append(prob_spam)
            ham_length_probs.append(prob_ham)
            counter += 1
        if counter > 6:
            break
        
    return spam_length_probs, ham_length_probs

def is_english_probs(list,count_normalizer,multipler,prob_not_eng,prob2):
    spam_is_english_probs = []
    ham_is_english_probs = []
    
    english_words = enchant.Dict("en_US")
    counter = 0
    prob_spam = prob_not_eng * multipler * count_normalizer
    prob_ham = (1 - prob_not_eng) * multipler
    for object in list:
        try:
            if not english_words.check(object):
                spam_is_english_probs.append(prob_spam)
                ham_is_english_probs.append(prob_ham)
                
        except:
            pass
        counter += 1
        if counter > 6:
            break
            
    for object in list:
        try:
            if english_words.check(object):
                spam_is_english_probs((1-prob2)*multipler*count_normalizer)
                ham_is_english_probs((prob2)*multipler)
        except:
            pass
        counter += 1
        if counter > 10:
            break
            
    return spam_is_english_probs, ham_is_english_probs


# DataFrame Based on Email List

In [1008]:
def get_email_list(ham_file_location, spam_file_location):
    
    all_files_ham = os.listdir(ham_file_location)
    ham_email_list = []
    ham_list = []
    ham_subject_list = []
    ham_split_list = []
    ham_subject_split_list = []
    ham_all_split_list = []
    ham_all = []
    
    for ham_email in all_files_ham:
        try:
            ham_email_txt = open(ham_file_location+"/" + ham_email).read()
        except:
            ham_email_txt = open(ham_file_location+"/" + ham_email,encoding='latin-1').read()
        
        ham_email_txt = ham_email_txt.replace('Subject: ','imzabatuhan928 ')
        
        for c in string.punctuation:
            ham_email_txt = ham_email_txt.replace(c,'')

        ham_email_txt = ham_email_txt.lower()
        
        subject_ham = re.findall("imzabatuhan928 (.*)",ham_email_txt)[0]
        
        ham_subject_list.append(subject_ham)
        ham_email_body = ham_email_txt.replace("imzabatuhan928"+ re.findall("imzabatuhan928(.*)"\
                                                                             ,ham_email_txt)[0],'')
        ham_list.append(ham_email_body)
        ham_email_list.append(ham_email)
        
        ham_email_txt_split = ham_email_body.split()
        ham_email_txt_split = list(Counter(ham_email_txt_split).keys())
        ham_split_list.append(ham_email_txt_split)
        
        subject_ham_split = subject_ham.split()
        subject_ham_split = list(Counter(subject_ham_split).keys())
        ham_subject_split_list.append(subject_ham_split)
        
                
        ham_email_txt = ham_email_txt.replace('imzabatuhan928','')
        ham_all.append(ham_email_txt)
        
        ham_email_split = ham_email_txt.split()
        ham_all_split_list.append(ham_email_split)
        
    df_ham = pd.DataFrame()
    df_ham['Email'] = ham_email_list
    df_ham['Email_All'] = ham_all
    df_ham['Email_All_Split'] = ham_all_split_list
    df_ham['Email_Body'] = ham_list
    df_ham['Email_Body_Split'] = ham_split_list
    df_ham['Email_Subject'] = ham_subject_list
    df_ham['Email_Subject_Split'] = ham_subject_split_list
    
    
    df_ham['F_Spam'] = 0

    
    #---------------------------------------
    
    
    all_files_spam = os.listdir(spam_file_location)
    spam_email_list = []
    spam_list = []
    spam_subject_list = []
    spam_split_list = []
    spam_subject_split_list = []
    spam_all_split_list = []
    spam_all = []
        
    for spam_email in all_files_spam:
        try:
            spam_email_txt = open(spam_file_location+"/" + spam_email).read()
        except:
            spam_email_txt = open(spam_file_location+"/" + spam_email,encoding='latin-1').read()
        
        
        spam_email_txt = spam_email_txt.replace('Subject: ','imzabatuhan928 ')
        
        for c in string.punctuation:
            spam_email_txt = spam_email_txt.replace(c,'')
         
        spam_email_txt = spam_email_txt.lower()

        subject_spam = re.findall("imzabatuhan928 (.*)",spam_email_txt)[0]
        
        spam_subject_list.append(subject_spam)
        spam_email_body = spam_email_txt.replace("imzabatuhan928"+ re.findall("imzabatuhan928(.*)"\
                                                                             ,spam_email_txt)[0],'')
        

        spam_list.append(spam_email_body)
        spam_email_list.append(spam_email)
        
        spam_email_txt_split = spam_email_body.split()
        spam_email_txt_split = list(Counter(spam_email_txt_split).keys())
        spam_split_list.append(spam_email_txt_split)
        
        subject_spam_split = subject_spam.split()
        subject_spam_split = list(Counter(subject_spam_split).keys())
        spam_subject_split_list.append(subject_spam_split)
        
                
        spam_email_txt = spam_email_txt.replace('imzabatuhan928','')
        spam_all.append(spam_email_txt)
        
        spam_email_split = spam_email_txt.split()
        spam_all_split_list.append(spam_email_split)
        
    df_spam = pd.DataFrame()
    df_spam['Email'] = spam_email_list
    df_spam['Email_All'] = spam_all
    df_spam['Email_All_Split'] = spam_all_split_list
    df_spam['Email_Body'] = spam_list
    df_spam['Email_Body_Split'] = spam_split_list
    df_spam['Email_Subject'] = spam_subject_list
    df_spam['Email_Subject_Split'] = spam_subject_split_list
    
    
    df_spam['F_Spam'] = 1
    
    
    
    df_all = pd.concat([df_spam,df_ham])
    
    df_all = df_all.reset_index()
    df_all = df_all.iloc[:,1:]
    
    
    X_spam = df_spam.drop('F_Spam',axis=1) 
    y_spam = df_spam['F_Spam'] 
    X_train_spam, X_test_spam, y_train_spam, y_test_spam = train_test_split(X_spam, y_spam\
                                                                            , test_size = 0.3, random_state = 0)
    
    X_ham = df_ham.drop('F_Spam',axis=1) 
    y_ham = df_ham['F_Spam'] 
    X_train_ham, X_test_ham, y_train_ham, y_test_ham = train_test_split(X_ham, y_ham, test_size = 0.3\
                                                                        , random_state = 0)
    
    X_train = pd.concat([X_train_spam,X_train_ham])
    X_test  = pd.concat([X_test_spam,X_test_ham])
    y_train = pd.concat([y_train_spam,y_train_ham])
    y_test  = pd.concat([y_test_spam,y_test_ham])
    
    X_train = X_train.reset_index()
    X_train = X_train.iloc[:,1:]
    X_test = X_test.reset_index()
    X_test = X_test.iloc[:,1:]    
    y_train = y_train.reset_index()
    y_train = y_train.iloc[:,1:]
    y_test = y_test.reset_index()
    y_test = y_test.iloc[:,1:]
    return df_all, X_train, X_test, y_train, y_test
    
    

# DataFrame Based on Word List

In [1013]:
def Get_Probiblities(X,Y,Delete_Words):
    
    X['F_Spam'] = Y
    
    #SPAM Mails
    
    Df_Spam = X[X['F_Spam']==1] 
    
    #get all words
    
    Spam_Email_Body = " ".join(Df_Spam['Email_Body'])
    Spam_Email_Subject = " ".join(Df_Spam['Email_Subject'])

    #Get list of all words
    
    Spam_Email_Body_List = Spam_Email_Body.split()
    Spam_Email_Subject_List = Spam_Email_Subject.split()
    
    Spam_Email_Body_List = remove_numbers(Spam_Email_Body_List)
    Spam_Email_Subject_List = remove_numbers(Spam_Email_Subject_List)

    #Get unique list of words and its counts 
    
    Spam_Email_Body_List_Unique = list(Counter(Spam_Email_Body_List).keys())
    Spam_Email_Subject_List_Unique = list(Counter(Spam_Email_Subject_List).keys())
    
    Spam_Email_Body_List_Count = list(Counter(Spam_Email_Body_List).values())
    Spam_Email_Subject_List_Count = list(Counter(Spam_Email_Subject_List).values())
    
    #Get words and its counts to DataFrame
    
    Df_Body_Words_Spam = pd.DataFrame()
    Df_Subject_Words_Spam = pd.DataFrame()
    
    Df_Body_Words_Spam['Words'] = Spam_Email_Body_List_Unique
    Df_Body_Words_Spam['Spam_Count'] = Spam_Email_Body_List_Count
    
    Df_Subject_Words_Spam['Words'] = Spam_Email_Subject_List_Unique
    Df_Subject_Words_Spam['Spam_Count'] = Spam_Email_Subject_List_Count
    
    #HAM Mails --------------- --------------- --------------- --------------- --------------- ---------------
     
    Df_Ham  = X[X['F_Spam']==0]
    
    #get all words
    
    Ham_Email_Body = " ".join(Df_Ham['Email_Body'])
    Ham_Email_Subject = " ".join(Df_Ham['Email_Subject'])
    
    #Get list of all words
    
    Ham_Email_Body_List = Ham_Email_Body.split()
    Ham_Email_Subject_List = Ham_Email_Subject.split()

    Ham_Email_Body_List = remove_numbers(Ham_Email_Body_List)
    Ham_Email_Subject_List = remove_numbers(Ham_Email_Subject_List)

    #Get unique list of words and its counts 
    
    Ham_Email_Body_List_Unique = list(Counter(Ham_Email_Body_List).keys())
    Ham_Email_Subject_List_Unique = list(Counter(Ham_Email_Subject_List).keys())
    
    Ham_Email_Body_List_Count = list(Counter(Ham_Email_Body_List).values())
    Ham_Email_Subject_List_Count = list(Counter(Ham_Email_Subject_List).values())
    
    Df_Body_Words_Ham = pd.DataFrame()
    Df_Subject_Words_Ham = pd.DataFrame()
        
    Df_Body_Words_Ham['Words'] = Ham_Email_Body_List_Unique
    Df_Body_Words_Ham['Ham_Count'] = Ham_Email_Body_List_Count
    
    Df_Subject_Words_Ham['Words'] = Ham_Email_Subject_List_Unique
    Df_Subject_Words_Ham['Ham_Count'] = Ham_Email_Subject_List_Count

    # JOIN TWO DFs
    
    Number_of_Spam_Email = len(Df_Spam)
    Number_of_Ham_Email = len(Df_Ham)
    Totatl_Email = Number_of_Spam_Email+Number_of_Ham_Email
    P_Spam = Number_of_Spam_Email/Totatl_Email
    P_Ham = Number_of_Ham_Email /Totatl_Email
    
    Df_Subject = Df_Subject_Words_Ham.merge(Df_Subject_Words_Spam, how='outer',on='Words')
    Df_Subject.fillna(0,inplace=True)
    Df_Subject['Total_Count'] = Df_Subject[['Ham_Count','Spam_Count']].sum(axis=1)
    Df_Subject = Df_Subject.sort_values('Total_Count', ascending=False)
    Df_Subject = Df_Subject.reset_index()
    Df_Subject = Df_Subject.iloc[:,1:]
    
    #------------------------------------------
    Df_Body = Df_Body_Words_Ham.merge(Df_Body_Words_Spam, how='outer',on='Words')
    Df_Body.fillna(0,inplace=True)
    Df_Body['Total_Count'] = Df_Body[['Ham_Count','Spam_Count']].sum(axis=1)
    Df_Body = Df_Body.sort_values('Total_Count', ascending=False)
    Df_Body = Df_Body.reset_index()
    Df_Body = Df_Body.iloc[:,1:]
    
    #------------------------------------------
    Df_Email = Df_Body.merge(Df_Subject, how='outer',on='Words')
    Df_Email.fillna(0,inplace=True)
    
    Df_Email['Ham_Count'] = Df_Email[['Ham_Count_x','Ham_Count_y']].sum(axis=1)
    Df_Email['Spam_Count'] = Df_Email[['Spam_Count_x','Spam_Count_y']].sum(axis=1)
    Df_Email['Total_Count'] = Df_Email[['Total_Count_x','Total_Count_y']].sum(axis=1)
    
    Df_Email['P(Word|Spam)'] = Df_Email['Spam_Count']/Df_Email['Spam_Count'].sum(axis=0)
    Df_Email['P(Word|Ham)']  = Df_Email['Ham_Count']/Df_Email['Ham_Count'].sum(axis=0)
    
    Df_Email[Df_Email['P(Word|Spam)']==0] = 0.1
    Df_Email[Df_Email['P(Word|Ham)']==0]  = 0.01
    
    dict = {'Words': ['enron', 'transmission', 'energy', 'powerplants', 'intercontinental', 'financial', 'workshop',
                      'retantion', 'article', 'fw', 'fyi', 'junaury', 'feburuary', 'march', 'april', 'may',
                      'june', 'july', 'august', 'september','october', 'november', 'december', 'pipe' ,'pipeline',
                     'xls','attached','txt','click','parchuse'],
            'Ham_Count': [ 100, 100, 100, 100, 100, 100, 100,
                      100, 100, 100, 100, 100 ,100, 100, 100, 100,
                      100, 100, 100, 100,100,100, 100, 100 ,100,100,100,100,1,1],
            'Spam_Count': [ 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1 ,1, 1, 1, 1,
                      1, 1, 1, 1,1,1, 1, 1 ,1,1,1,1,100,100],
            'Total_Count': [ 101, 101, 101, 101, 101, 101, 101,
                      101, 101, 101, 101, 101 ,101, 101, 101, 101,
                      101, 101, 101, 101, 101, 101, 101, 101 ,101,101,101,101,101,101],
            'P(Word|Spam)': [ 1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam,
                      1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam ,1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam,
                      1*P_Spam, 1*P_Spam, 1*P_Spam, 1*P_Spam,1*P_Spam,1*P_Spam, 1*P_Spam, 1*P_Spam ,1*P_Spam
                             , 1*P_Spam ,1*P_Spam,1*P_Spam,1000*P_Spam,1000*P_Spam],
            'P(Word|Ham)': [ 1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham,
                      100*P_Ham, 100*P_Ham, 100*P_Ham, 1000*P_Ham, 1000*P_Ham ,1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham,
                      1000*P_Ham, 1000*P_Ham, 1000*P_Ham, 1000*P_Ham,1000*P_Ham,1000*P_Ham, 1000*P_Ham, 1000*P_Ham ,1000*P_Ham
                            , 1000*P_Ham ,1000*P_Ham,1000*P_Ham,1*P_Ham,1*P_Ham]
            }
    
    dict_df = pd.DataFrame(dict)
    
    Df_Email['P(Word|Spam)'] = Df_Email['P(Word|Spam)']*1000
    Df_Email['P(Word|Ham)'] = Df_Email['P(Word|Ham)']*1000
    
    
    Df_Email.drop(['Spam_Count_x','Spam_Count_y','Ham_Count_x','Ham_Count_y','Total_Count_x','Total_Count_y'\
                  ],axis=1,inplace=True)
    
    Df_Email = Df_Email[~Df_Email['Words'].isin(Delete_Words)]
    Df_Email = Df_Email[Df_Email['Words'].str.len()>2]
    
    Df_Email = pd.concat([Df_Email,dict_df])
   
    Df_Email = Df_Email.sort_values('Total_Count', ascending=False)
    Df_Email = Df_Email.reset_index()
    Df_Email = Df_Email.iloc[:,1:]
    
    return Df_Body, Df_Subject, Df_Email, P_Spam, P_Ham

In [1036]:
def Fit_Naive_Bayes_Model(X,Y,DF,P_Spam,P_Ham,Delete_Words):  
    
    Spam_Probs = []
    spam_count = DF['Spam_Count'].sum()
    ham_count = DF['Ham_Count'].sum()
    count_normalizer = spam_count/ham_count
    
    for index, row in X.iterrows():
        spam_list_of_in = []
        spam_list_of_not_in = []
        ham_list_of_not_in = []
        ham_list_of_in = []
        
        
        row['Email_All_Split'] = list(set(row['Email_All_Split']) - set(Delete_Words))
        row['Email_All_Split'] = remove_length(row['Email_All_Split'])
        row['Email_All_Split'] = remove_numbers(row['Email_All_Split'])
        
        
        
        spam_length_probs, ham_length_probs = length_probs(row['Email_All_Split'],count_normalizer,60,0.9)
        spam_is_english_probs, ham_is_english_probs = is_english_probs(row['Email_All_Split'],count_normalizer\
                                                                        ,60,0.85,0.55)
        
        spam_list = list(DF[DF['Words'].isin(row['Email_All_Split'])]['P(Word|Spam)'])
        spam_list.extend(spam_length_probs)
        spam_list.extend(spam_is_english_probs)

        P_Spam_M = np.prod(spam_list) * P_Spam
        
        ham_list = list(DF[DF['Words'].isin(row['Email_All_Split'])]['P(Word|Ham)'])
        ham_list.extend(ham_length_probs)
        ham_list.extend(ham_is_english_probs)
        P_Ham_M  = np.prod(ham_list) * P_Ham

        
        P_Spam_M_Norm = P_Spam_M / (P_Spam_M + P_Ham_M)
        Spam_Probs.append(P_Spam_M_Norm)      

    X['Real'] = Y
    X['Pred'] = Spam_Probs
    X['1'] = 1
    X['0'] = 0
    X['Pred'].fillna(0,inplace=True)
    
    return X[['Real','Pred','1','0']]

## Load Files

In [936]:


df_all_6, X_train_6, X_test_6, y_train_6, y_test_6 = get_email_list("enron6/ham", "enron6/spam")
Df_Body_6, Df_Subject_6, Df_Email_6, P_Spam_6, P_Ham_6 = Get_Probiblities(X_train_6, y_train_6, Delete_Words)

df_all_1, X_train_1, X_test_1, y_train_1, y_test_1 = get_email_list("enron1/ham", "enron1/spam")
Df_Body_1, Df_Subject_1, Df_Email_1, P_Spam_1, P_Ham_1 = Get_Probiblities(X_train_1, y_train_1, Delete_Words)

In [986]:
X_test_1.loc[861,:]

,Email,Email_All,Email_All_Split,Email_Body,Email_Body_Split,Email_Subject,Email_Subject_Split,Real,Pred,1,0
861,4792.2005-07-02.GP.spam.txt,healthy reproductive analysis\ni just wanted ...,"[really, reproductive, man, good, also, counte...",\ni just wanted to write and thank you for spu...,"[i, just, wanted, to, write, and, thank, you, ...",healthy reproductive analysis,"[healthy, reproductive, analysis]",1,1.000000e+00,1,0
861,0560.2000-03-06.farmer.ham.txt,manager coaching program\nd fyi \n ...,"[call, , , com, contact, week, program, staff,...",\nd fyi \n forwarded by...,"[d, fyi, forwarded, by, brenda, f, herod, hou,...",manager coaching program,"[manager, coaching, program]",0,3.517466e-28,1,0


In [864]:
Df_Email_6.head(5)

,Words,Ham_Count,Spam_Count,Total_Count,P(Word|Spam),P(Word|Ham)
0,company,668.0,6090.0,6758.0,4.239559,1.260159
1,com,1976.0,3818.0,5794.0,2.657904,3.727655
2,please,1590.0,2912.0,4502.0,2.027192,2.999479
3,information,1142.0,3324.0,4466.0,2.314006,2.154343
4,statements,16.0,3974.0,3990.0,2.766504,0.030183


## For--> Train: Enron-1 , Test: Enron-1

In [1047]:
DF = Fit_Naive_Bayes_Model(X_test_1,y_test_1,Df_Email_1,P_Spam_1,P_Ham_1,Delete_Words)
True_Positive =  (DF[(DF['Pred'] >= 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
False_Negative =  (DF[(DF['Pred'] < 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
True_Negative = (DF[(DF['Pred'] < 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
False_Positive = (DF[(DF['Pred'] >= 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
print("For--> Train: Enron-1 , Test: Enron-1")
Precision = True_Positive/(True_Positive+False_Positive)
Recall = True_Positive/(True_Positive+False_Negative)
F_1 = (2*Precision*Recall)/ (Precision + Recall)
print("True Positive: %0.2f"%True_Positive)
print("True Negative: %0.2f"%True_Negative)
print("False Negative: %0.2f"%False_Negative)
print("False Positive: %0.2f"%False_Positive)
print("Precision: %0.2f"%Precision)
print("Recall: %0.2f"%Recall)
print("F 1: %0.2f"%F_1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


For--> Train: Enron-1 , Test: Enron-1
True Positive: 0.95
True Negative: 0.96
False Negative: 0.05
False Positive: 0.04
Precision: 0.96
Recall: 0.95
F 1: 0.95


## For--> Train: Enron-6 , Test: Enron-6

In [1046]:
DF = Fit_Naive_Bayes_Model(X_test_6,y_test_6,Df_Email_6,P_Spam_6,P_Ham_6,Delete_Words)
True_Positive =  (DF[(DF['Pred'] >= 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
False_Negative =  (DF[(DF['Pred'] < 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
True_Negative = (DF[(DF['Pred'] < 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
False_Positive = (DF[(DF['Pred'] >= 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
print("For--> Train: Enron-6 , Test: Enron-6")
Precision = True_Positive/(True_Positive+False_Positive)
Recall = True_Positive/(True_Positive+False_Negative)
F_1 = (2*Precision*Recall)/ (Precision + Recall)
print("True Positive: %0.2f"%True_Positive)
print("True Negative: %0.2f"%True_Negative)
print("False Negative: %0.2f"%False_Negative)
print("False Positive: %0.2f"%False_Positive)
print("Precision: %0.2f"%Precision)
print("Recall: %0.2f"%Recall)
print("F 1: %0.2f"%F_1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


For--> Train: Enron-6 , Test: Enron-6
True Positive: 0.91
True Negative: 0.97
False Negative: 0.09
False Positive: 0.03
Precision: 0.97
Recall: 0.91
F 1: 0.94


## For--> Train: Enron-1 , Test: Enron-6

In [1045]:
DF = Fit_Naive_Bayes_Model(X_test_6,y_test_6,Df_Email_1,P_Spam_1,P_Ham_1,Delete_Words)
True_Positive =  (DF[(DF['Pred'] >= 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
False_Negative =  (DF[(DF['Pred'] < 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
True_Negative = (DF[(DF['Pred'] < 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
False_Positive = (DF[(DF['Pred'] >= 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
print("For--> Train: Enron-1 , Test: Enron-6")
Precision = True_Positive/(True_Positive+False_Positive)
Recall = True_Positive/(True_Positive+False_Negative)
F_1 = (2*Precision*Recall)/ (Precision + Recall)
print("True Positive: %0.2f"%True_Positive)
print("True Negative: %0.2f"%True_Negative)
print("False Negative: %0.2f"%False_Negative)
print("False Positive: %0.2f"%False_Positive)
print("Precision: %0.2f"%Precision)
print("Recall: %0.2f"%Recall)
print("F 1: %0.2f"%F_1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


For--> Train: Enron-1 , Test: Enron-6
True Positive: 0.96
True Negative: 0.91
False Negative: 0.04
False Positive: 0.09
Precision: 0.92
Recall: 0.96
F 1: 0.94


## For--> Train: Enron-6 , Test: Enron-1

In [1044]:
DF = Fit_Naive_Bayes_Model(X_test_1,y_test_1,Df_Email_6,P_Spam_6,P_Ham_6,Delete_Words)
True_Positive =  (DF[(DF['Pred'] >= 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
False_Negative =  (DF[(DF['Pred'] < 0.5) & (DF['Real']==1)]['1'].sum()/DF[DF['Real'] >= 0.5]['1'].sum())
True_Negative = (DF[(DF['Pred'] < 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
False_Positive = (DF[(DF['Pred'] >= 0.5) & (DF['Real']==0)]['1'].sum()/DF[DF['Real'] < 0.5]['1'].sum())
print("For--> Train: Enron-6 , Test: Enron-1")
Precision = True_Positive/(True_Positive+False_Positive)
Recall = True_Positive/(True_Positive+False_Negative)
F_1 = (2*Precision*Recall)/ (Precision + Recall)
print("True Positive: %0.2f"%True_Positive)
print("True Negative: %0.2f"%True_Negative)
print("False Negative: %0.2f"%False_Negative)
print("False Positive: %0.2f"%False_Positive)
print("Precision: %0.2f"%Precision)
print("Recall: %0.2f"%Recall)
print("F 1: %0.2f"%F_1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


For--> Train: Enron-6 , Test: Enron-1
True Positive: 0.87
True Negative: 0.97
False Negative: 0.13
False Positive: 0.03
Precision: 0.96
Recall: 0.87
F 1: 0.91
